In [1]:
import chess
import chess.pgn
import ipywidgets as widgets
import chess.svg
from IPython.display import display, clear_output
import time
from versions.my_engine_v1 import get_best_move
# from stockfish import Stockfish


In [2]:

board = chess.Board()
AI_DEPTH = 3


In [3]:

def pgn_viewer(game: chess.pgn.Game):
    board = game.board()
    moves = list(game.mainline_moves())

    slider = widgets.IntSlider(
        value=0,
        min=0,
        max=len(moves),
        step=1,
        description="Move",
        continuous_update=False
    )

    out = widgets.Output()

    def render(move_idx):
        board.reset()
        for i in range(move_idx):
            board.push(moves[i])
        return chess.svg.board(board, size=420)

    def on_change(change):
        with out:
            out.clear_output(wait=True)
            display(render(change["new"]))

    slider.observe(on_change, names="value")

    # initial render
    with out:
        display(render(0))

    controls = widgets.HBox([
        widgets.Button(description="⏮", layout=widgets.Layout(width="50px")),
        widgets.Button(description="◀", layout=widgets.Layout(width="50px")),
        slider,
        widgets.Button(description="▶", layout=widgets.Layout(width="50px")),
        widgets.Button(description="⏭", layout=widgets.Layout(width="50px")),
    ])

    def go_start(_): slider.value = 0
    def go_prev(_): slider.value = max(0, slider.value - 1)
    def go_next(_): slider.value = min(len(moves), slider.value + 1)
    def go_end(_): slider.value = len(moves)

    controls.children[0].on_click(go_start)
    controls.children[1].on_click(go_prev)
    controls.children[3].on_click(go_next)
    controls.children[4].on_click(go_end)

    display(controls, out)


In [ ]:


# FIX 1: Pass claim_draw=True to stop on 3-fold repetition or 50-move rule
def play(claim_draw=True,board=board):
    game = chess.pgn.Game()
    game.headers["Event"] = "AI vs AI"
    game.headers["Site"] = "Local"
    node = game 
    
    start_time = time.perf_counter()

    while not board.is_game_over(claim_draw=claim_draw):
        if board.turn == chess.WHITE:
            move = get_best_move(board, AI_DEPTH)
        else:
            move = get_best_move(board, AI_DEPTH)

        board.push(move)
        node = node.add_variation(move)

        # Visualization
        clear_output(wait=True)
        display(chess.svg.board(board, size=400))

    end_time = time.perf_counter()

    result = board.result(claim_draw=True)
    game.headers["Result"] = result
    print(f"Outcome: {board.outcome(claim_draw=True)}")
    print(f"\ntime taken {end_time-start_time} PGN Output: ",game)
    pgn_viewer(game)
    board.reset_board()

play()

In [ ]:
""" 
sf = Stockfish(path=".venv/Lib/site-packages/stockfish",depth=3)

def play_vs_stockfish():
    board = chess.Board()
    while not board.is_game_over():
        if board.turn == chess.WHITE:
            move = get_best_move(board, 3)
        else:
            sf.set_fen_position(board.fen())
            move = chess.Move.from_uci(sf.get_best_move())

        board.push(move)

    print(board.result())
"""
